In [463]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [464]:
actuals = pd.read_csv('D:\projects\data\Actual-2020 December.csv')

c:\users\rbenj\appdata\local\programs\python\python37\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [465]:
actuals[actuals['id'] == 1159]

,id,timeFrom,energyKwh
93807,1159,2020-12-31 22:00:00,31.376


In [466]:
print(actuals.isna().sum().sum())
print((actuals==0).sum().sum())

# meaning no null values, but alot of 0 values (is it possible?)

0
27150


In [467]:

# lets group by station
pt_by_station = pd.pivot_table(data = actuals, values = 'energyKwh',index = 'timeFrom', columns='id',fill_value=0)
pt_by_station['total'] = pt_by_station.sum(axis=1)
pt_by_station *= 2
pt_by_station.index = pd.to_datetime(pt_by_station.index)

In [468]:
pt_by_station['total'].max()


509341.5602712

In [527]:
daily_max = pt_by_station.groupby(by=pt_by_station.index.day).max()
daily_max.apply(np.std,axis=0)[:-1].sort_values()

id
774                              0.000000
268                              0.000000
239                              0.000000
1057                             0.000000
770                              0.000000
772                              0.000000
778                              0.000000
784                              0.000000
786                              0.000000
790                              0.000000
396                              0.000000
797                              0.000000
792                              0.000000
782                              0.027527
939                              0.070343
1058                             0.556630
820                              0.971933
568                              1.063825
944                              2.320540
1052                             2.371907
794                              2.647838
788                              2.687394
1055                             2.824655
241                            

In [470]:
# the following is a list of station ranked by the number of zero values they have
stations_by_zero_values = (pt_by_station==0).sum(axis=0).sort_values(ascending=False)

# we would like to take out of our calculation any station that has more
# than 500 zero values, as it would inappropriately be marked as 'safe'

uncalculated_meters = stations_by_zero_values[stations_by_zero_values>500].index

In [471]:
# lets get the must meters:
meters_used = actuals['id'].unique()
meters = pd.read_csv(f'..\\data\\OPC-Meters.csv')
must_customers = pd.read_csv(f'..\\data\\must_meters.csv')
must_station = [station for station in meters['name'].values for must in must_customers['Must Customers'].values if must in station]
must_meters = meters[meters['name'].isin(must_station)]['meterId'].values
must_meters_used = [meter for meter in must_meters if meter in meters_used]

In [472]:
must_meters_used

['Gadot Main',
 'Hadera Paper Main',
 'Intel F28',
 'Intel IDC1-4',
 'Intel IDC9',
 'Intel IDPj',
 '92',
 'Iscar MainTools',
 '93',
 '91',
 '95',
 'ORL']

In [473]:
pt_by_station_to_rank = pt_by_station.drop(columns=must_meters_used).drop(columns=uncalculated_meters)

In [532]:
daily_max = pt_by_station_to_rank.groupby(by=pt_by_station_to_rank.index.day).max()
daily_max_std = daily_max.apply(np.std,axis=0)[:-1].sort_values()

daily_max.to_csv('daily_maxes.csv',header=True)

In [530]:
daily_max_std

id
939                              0.070343
568                              1.063825
944                              2.320540
1052                             2.371907
794                              2.647838
788                              2.687394
1055                             2.824655
241                              2.879494
908                              2.999068
780                              3.072076
776                              3.441799
1059                             3.579506
1063                             3.860077
1028                             3.929234
1064                             4.001072
1048                             4.041149
795                              4.124220
566                              4.444581
574                              5.105698
1027                             5.221763
1047                             5.360208
585                              5.493479
1029                             5.531806
819                            

In [ ]:
# now lets rate the station by the amount they contribute to the total maximum


effect_by_meter = {}
for column in pt_by_station_to_rank.columns:
    max_without = (pt_by_station_to_rank['total']-pt_by_station_to_rank[column]).max() 
    effect_on_max = pt_by_station_to_rank['total'].max() - max_without
    effect_by_meter[column] = effect_on_max

# and rate them by that risk    
meters_by_risk = pd.Series(effect_by_meter).sort_values()[:-1]

In [ ]:
# doing the same for total consumption might also be right, that way 
# we can measure risk against importance again

meters_by_consumption = pt_by_station_to_rank.sum()
stations_risk_cons = pd.DataFrame({'effect_on_max':pd.Series(effect_by_meter),'consumption':meters_by_consumption})

In [ ]:
stations_risk_cons['ratio'] = stations_risk_cons['effect_on_max'] /stations_risk_cons['consumption']  

stations_risk_cons.sort_values('ratio',ascending=False).to_csv('station_risk_consumption.csv')

In [ ]:
rate = 300
meters_included = must_meters_used[:]
for meter in meters_by_risk.index:
    current_max = pt_by_station[meters_included].sum(axis=1).max()
    if current_max > rate*1000:
        break
    meters_included.append(meter)
    


#pt_by_station[meters_included]
len(meters_included)

In [525]:
# doing the same for total consumption might also be right, that way 
# we can measure risk against importance again

meters_by_consumption = pt_by_station_to_rank.sum()
stations_risk_cons = pd.DataFrame({'effect_on_max':pd.Series(effect_by_meter),'consumption':meters_by_consumption})

In [526]:
stations_risk_cons['ratio'] = stations_risk_cons['effect_on_max'] /stations_risk_cons['consumption']  

stations_risk_cons.sort_values('ratio',ascending=False).to_csv('station_risk_consumption.csv')

In [503]:
rate = 300
meters_included = must_meters_used[:]
for meter in meters_by_risk.index:
    current_max = pt_by_station[meters_included].sum(axis=1).max()
    if current_max > rate*1000:
        break
    meters_included.append(meter)
    


#pt_by_station[meters_included]
len(meters_included)

279